In [3]:
from llama_generation import LLAMA_Generation

import os

os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12357'
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'

agent = LLAMA_Generation(
        ckpt_dir='llama-2-13b-chat/',
        tokenizer_path='tokenizer.model',
        max_seq_len=512 * 2,
        max_batch_size=8
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 50.00 MiB. GPU 0 has a total capacity of 23.62 GiB of which 81.75 MiB is free. Including non-PyTorch memory, this process has 22.64 GiB memory in use. Of the allocated memory 22.21 GiB is allocated by PyTorch, and 3.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# judge professorship

In [ ]:
agent.instructions_text

In [2]:
# content_text =\
#     "Changhwan Kim (Member, IEEE) received the B.S. degree in mechanical engineering and the M.S. degree in machine design engineering from Hanyang University, Seoul, South Korea, in 1993 and 1995, respectively, and the Ph.D. degree in mechanical engineering from The University of Iowa, Iowa City, IA, USA, in 2002. From 2002 to 2004, he was a Research Associate with the Robotics and Automation Laboratory, University of Notre Dame, Notre Dame, IN, USA. Since 2004, he has been working at the Korea Institute of Science and Technology (KIST), Seoul. His research interests include task and motion planning for robot manipulation and social robots.(Based on document published on 26 December 2022)."


content_text =\
"""
Google Affiliation: The BioRobotics Institute, Scuola Superiore Sant'Anna 
IEEE Biography: 
Andrea Mariani, The BioRobotics Institute and Department of Excellence in Robotics and AI, Sant’Anna School of Advanced Studies, Pisa, 56025, Italy. 
E-mail: andrea.mariani@santannapisa.it.(Based on document published on 1 August 2022).
"""

# content_text = "N/A"
# content_text = "Yan Zhuang (Senior Member, IEEE) is not a professor"

import time
start = time.time()

response, original_answer = agent.judge_professorship(content_text, verbose=False)


# remove space in string
# response = response.replace(" ", "")
print(response)
print(original_answer)
print(time.time() - start)

YES
 Based on the information provided, I would respond with "YES" as Changhwan Kim is listed as a Professor at the Korea Institute of Science and Technology (KIST) in Seoul, South Korea. His titles and positions mentioned in the document include:
* Member, IEEE
* B.S. degree in mechanical engineering, M.S. degree in machine design engineering, and Ph.D. degree in mechanical engineering from various universities
* Research Associate, Robotics and Automation Laboratory, University of Notre Dame, Notre Dame, IN, USA (2002-2004)
* Professor, Korea Institute of Science and Technology (KIST), Seoul, South Korea (since 2004)
Therefore, based on the information provided, Changhwan Kim holds the position of a Professor at KIST.
3.963902473449707


In [ ]:
import json
with open(f'auther_database_v5_5_2_extended_no_score.json', 'r') as file:
    author_database = json.load(file)


import time

min_papers = 10 
count = 0

for pid, author in author_database.items():
    
    # if 'robor_affiliation' not in author and ["is_professor"] == 'None':
    if 'is_professor' in author:
        if author['is_professor'] == 'None' and author['number_of_ieee_paper'] >10:
        # if author['is_professor'] == 'None' and author['num_papers'] >10:
        # if author['is_professor'] == 'None' and author['number_of_ieee_paper'] >10 and 'ieee_affiliation' not in author:
            
            print()
            print(pid)
            print(author['number_of_ieee_paper'])
            print(author['ieee_affiliation'])
            print(author['ieee_biography'])

            if author['ieee_biography'] == 'N/A':
                response = 'NO'
                original_answer = 'N/A'
            else:
                response,original_answer = agent.judge_professorship(author['ieee_biography'], verbose=False)

            start = time.time()
            # remove space in string
            # response = response.replace(" ", "")
            print(response)
            print(original_answer)
            print(time.time() - start)

            count += 1

            if count > 10:
                break

print(count)
# print(f" > {min_papers} papers \t {count}")


# with open(f'auther_database_v5_5_1_extended_no_score.json', 'w') as file:
#     json.dump(author_database, file, indent=4)

# Find affiliation

In [ ]:
# read csv to string

# with open('country-info3_3.csv', 'r') as file:
#     author_database = json.load(file)

import csv

file_path = 'country-info3_3.csv'  # Replace this with your file path

with open(file_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)  # Skip the header row
    first_column_values = [row[0] for row in reader if row]  # Check if row is not empty
    csv_string = '\n'.join(first_column_values)


instruct_text = \
    "Based on the given information, please provide which of the following institute is the people belong to. " \
    "If the information is not enough to determine the institute, please provide the answer as 'N/A'.\n" + \
csv_string

agent.instructions_text = instruct_text

content_text = "'Centre for Advanced Robotics Technology Innovation (CARTIN)', 'School of Electrical and Electronic Engineering', 'Nanyang Technological University, Singapore'"


affiliation = agent.judge_affiliation(content_text, verbose=False)

print(affiliation)


# delete the agent and empty the GPU memory

In [ ]:
del agent
import gc
gc.collect()
# clena the GPU memory
import torch
torch.cuda.empty_cache()